In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
import numpy as np 
import pandas as pd 
import seaborn as sns
!pip install gensim spacy nltk
import re 
import gensim
import spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
data= pd.read_csv('/content/drive/My Drive/Bachelor/Cleaned Data/Sarcasm.csv') 

In [4]:
import numpy as np
max_sequence_length = np.max(data['Number of Words in tweet'])
print("max length is:", max_sequence_length)

max length is: 56


In [5]:
text = data['cleaned_tweet']
label = data['sarcasm']

In [6]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(text.astype(str))
sequences = tokenizer.texts_to_sequences(text.astype(str))

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))


from keras_preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Found 55395 unique tokens.
Shape of data tensor: (26095, 56)


In [7]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(label))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (26095, 2)


In [8]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

# Arabert

In [9]:
!pip install bert-for-tf2
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=bb41fc5214661cdff1169e70f41e844028675151ad1084554e56f2e2edd21ac2
  Stored in directory: /root/.cache/pip/wheels/ab/a4/72/df07592cea3ae06b5e846f5e52262f8b16748e829ca354b7df
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=c2fa72c49e1b805d83b90b09d9c4806128a3325f8e62117777ddfc40eed05bb8
  Stored in directory: /root/.cache/pip/wheels/c7/f3/85/b8cf1d8bfe55dc2ece0f1fcd4e91d6f8fc7b59ff3fd75329e1
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=d00a191b25fb73d8db95245bf30aa5defd09ebb2b4d4da342e0efd4feea4f09e
  Sto

In [10]:
import numpy as np
import math
import re
import pandas as pd
import random

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt

In [11]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
from transformers import AutoTokenizer, AutoModel

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [12]:
arabert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv01",do_lower_case=False)
def encode_sentence(sent):
    tokenized = arabert_tokenizer.tokenize(sent)
    return arabert_tokenizer.convert_tokens_to_ids(tokenized)

Downloading:   0%|          | 0.00/379 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

# RNN-BiLSTM

In [ ]:
# RNN layer followed by a BiLSTM layer and also a Softmax activation function for classification, with an Adam optimizer

from tensorflow.keras import layers
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 61s 94ms/step - loss: 0.4584 - accuracy: 0.8216
Epoch 2/10
612/612 [==============================] - 40s 66ms/step - loss: 0.2633 - accuracy: 0.9009
Epoch 3/10
612/612 [==============================] - 50s 81ms/step - loss: 0.1335 - accuracy: 0.9551
Epoch 4/10
612/612 [==============================] - 69s 113ms/step - loss: 0.0764 - accuracy: 0.9761
Epoch 5/10
612/612 [==============================] - 68s 111ms/step - loss: 0.0489 - accuracy: 0.9863
Epoch 6/10
612/612 [==============================] - 50s 81ms/step - loss: 0.0388 - accuracy: 0.9897
Epoch 7/10
612/612 [==============================] - 40s 65ms/step - loss: 0.0322 - accuracy: 0.9916
Epoch 8/10
612/612 [==============================] - 41s 67ms/step - loss: 0.0292 - accuracy: 0.9920
Epoch 9/10
612/612 [==============================] - 39s 64ms/step - loss: 0.0256 - accuracy: 0.9923
Epoch 10/10
612/612 [==============================] - 39s 63ms/step - loss: 0.0223 - accur

In [ ]:
predictions= model.predict(x_test)
predictions=np.argmax(predictions, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions))
cm = confusion_matrix(y_test, predictions)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions))
print('F1 Score: %.3f' % f1_score(y_test, predictions))
print('Precision: %.3f' % precision_score(y_test, predictions))

204/204 [==============================] - 2s 8ms/step
Recall: 0.929
The Confusion Matrix is [[5276   82]
 [  83 1082]]
Accuracy: 0.975
F1 Score: 0.929
Precision: 0.930


In [ ]:
#RNN - BiLSTM with flatten 

from tensorflow.keras import layers
model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model1.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model1.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 42s 64ms/step - loss: 0.4597 - accuracy: 0.8219
Epoch 2/10
612/612 [==============================] - 39s 64ms/step - loss: 0.2706 - accuracy: 0.8964
Epoch 3/10
612/612 [==============================] - 39s 63ms/step - loss: 0.1337 - accuracy: 0.9543
Epoch 4/10
612/612 [==============================] - 41s 67ms/step - loss: 0.0752 - accuracy: 0.9762
Epoch 5/10
612/612 [==============================] - 39s 64ms/step - loss: 0.0517 - accuracy: 0.9848
Epoch 6/10
612/612 [==============================] - 39s 64ms/step - loss: 0.0411 - accuracy: 0.9891
Epoch 7/10
612/612 [==============================] - 39s 64ms/step - loss: 0.0338 - accuracy: 0.9905
Epoch 8/10
612/612 [==============================] - 41s 67ms/step - loss: 0.0305 - accuracy: 0.9911
Epoch 9/10
612/612 [==============================] - 39s 63ms/step - loss: 0.0253 - accuracy: 0.9918
Epoch 10/10
612/612 [==============================] - 39s 64ms/step - loss: 0.0231 - accurac

In [ ]:
predictions1= model1.predict(x_test)
predictions1=np.argmax(predictions1, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions1))
cm = confusion_matrix(y_test, predictions1)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions1))
print('F1 Score: %.3f' % f1_score(y_test, predictions1))
print('Precision: %.3f' % precision_score(y_test, predictions1))

204/204 [==============================] - 2s 8ms/step
Recall: 0.917
The Confusion Matrix is [[5267   85]
 [  97 1074]]
Accuracy: 0.972
F1 Score: 0.922
Precision: 0.927


# Learning Rates & Optimizers Tuning

In [ ]:
#Model 5: 1  RNN - BiLSTM with flatten layer, Adam 1e-3

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model14 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model14.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model14.fit(x_train, y_train, batch_size=32, epochs=10)


accuracy14= model14.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy14)


predictions14= model14.predict(x_test)
predictions14=np.argmax(predictions14, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions14))
cm = confusion_matrix(y_test, predictions14)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions14))
print('F1 Score: %.3f' % f1_score(y_test, predictions14))
print('Precision: %.3f' % precision_score(y_test, predictions14))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 60s 79ms/step - loss: 0.3924 - accuracy: 0.8430
Epoch 2/10
612/612 [==============================] - 66s 108ms/step - loss: 0.1421 - accuracy: 0.9504
Epoch 3/10
612/612 [==============================] - 45s 74ms/step - loss: 0.0366 - accuracy: 0.9889
Epoch 4/10
612/612 [==============================] - 46s 75ms/step - loss: 0.0177 - accuracy: 0.9927
Epoch 5/10
612/612 [==============================] - 64s 104ms/step - loss: 0.0121 - accuracy: 0.9941
Epoch 6/10
612/612 [==============================] - 49s 80ms/step - loss: 0.0167 - accuracy: 0.9927
Epoch 7/10
612/612 [==============================] - 62s 101ms/step - loss: 0.0122 - accuracy: 0.9931
Epoch 8/10
612/612 [==============================] - 68s 111ms/step - loss: 0.0100 - accuracy: 0.9940
Epoch 9/10
612/612 [==============================] - 52s 85ms/step - loss: 0.0089 - accuracy: 0.9941
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.6174 - accur

In [ ]:
#Model 5: 1  RNN - BiLSTM with flatten layer, Adam 1e-2

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model15 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model15.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-2),
              metrics=['accuracy'])

model15.fit(x_train, y_train, batch_size=32, epochs=10)


accuracy15= model15.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy15)


predictions15= model15.predict(x_test)
predictions15=np.argmax(predictions15, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions15))
cm = confusion_matrix(y_test, predictions15)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions15))
print('F1 Score: %.3f' % f1_score(y_test, predictions15))
print('Precision: %.3f' % precision_score(y_test, predictions15))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 41s 62ms/step - loss: 0.4325 - accuracy: 0.8275 - get_f1: 0.8275
Epoch 2/10
612/612 [==============================] - 38s 62ms/step - loss: 0.3221 - accuracy: 0.8664 - get_f1: 0.8663
Epoch 3/10
612/612 [==============================] - 38s 62ms/step - loss: 0.2603 - accuracy: 0.8940 - get_f1: 0.8941
Epoch 4/10
612/612 [==============================] - 39s 63ms/step - loss: 0.2127 - accuracy: 0.9188 - get_f1: 0.9188
Epoch 5/10
612/612 [==============================] - 38s 63ms/step - loss: 0.1787 - accuracy: 0.9329 - get_f1: 0.9330
Epoch 6/10
612/612 [==============================] - 39s 63ms/step - loss: 0.1552 - accuracy: 0.9425 - get_f1: 0.9425
Epoch 7/10
612/612 [==============================] - 43s 70ms/step - loss: 0.1392 - accuracy: 0.9484 - get_f1: 0.9484
Epoch 8/10
612/612 [==============================] - 40s 65ms/step - loss: 0.1295 - accuracy: 0.9550 - get_f1: 0.9550
Epoch 9/10
612/612 [==============================] - 49s 8

In [ ]:
#Model 5: 1  RNN - BiLSTM with flatten layer, Adam 1e-1

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model16 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model16.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-1),
              metrics=['accuracy'])

model16.fit(x_train, y_train, batch_size=32, epochs=10)


accuracy16= model16.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy16)


predictions16= model16.predict(x_test)
predictions16=np.argmax(predictions16, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions16))
cm = confusion_matrix(y_test, predictions16)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions16))
print('F1 Score: %.3f' % f1_score(y_test, predictions16))
print('Precision: %.3f' % precision_score(y_test, predictions16))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 43s 64ms/step - loss: 0.4970 - accuracy: 0.8175 - get_f1: 0.8176
Epoch 2/10
612/612 [==============================] - 39s 64ms/step - loss: 0.4696 - accuracy: 0.8222 - get_f1: 0.8223
Epoch 3/10
612/612 [==============================] - 44s 72ms/step - loss: 0.4694 - accuracy: 0.8222 - get_f1: 0.8222
Epoch 4/10
612/612 [==============================] - 39s 64ms/step - loss: 0.4693 - accuracy: 0.8222 - get_f1: 0.8222
Epoch 5/10
612/612 [==============================] - 39s 64ms/step - loss: 0.4703 - accuracy: 0.8222 - get_f1: 0.8222
Epoch 6/10
612/612 [==============================] - 39s 63ms/step - loss: 0.4695 - accuracy: 0.8222 - get_f1: 0.8223
Epoch 7/10
612/612 [==============================] - 39s 64ms/step - loss: 0.4695 - accuracy: 0.8222 - get_f1: 0.8223
Epoch 8/10
612/612 [==============================] - 39s 63ms/step - loss: 0.4691 - accuracy: 0.8222 - get_f1: 0.8222
Epoch 9/10
612/612 [==============================] - 39s 6

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Model 5: 1  RNN - BiLSTM with flatten layer, RMSprop 1e-4

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model17 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model17.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-4),
              metrics=['accuracy'])

model17.fit(x_train, y_train, batch_size=32, epochs=10)


accuracy17= model17.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy17)


predictions17= model17.predict(x_test)
predictions17=np.argmax(predictions17, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions17))
cm = confusion_matrix(y_test, predictions17)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions17))
print('F1 Score: %.3f' % f1_score(y_test, predictions17))
print('Precision: %.3f' % precision_score(y_test, predictions17))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 40s 60ms/step - loss: 0.4701 - accuracy: 0.8187
Epoch 2/10
612/612 [==============================] - 37s 60ms/step - loss: 0.3812 - accuracy: 0.8384
Epoch 3/10
612/612 [==============================] - 36s 59ms/step - loss: 0.3193 - accuracy: 0.8701
Epoch 4/10
612/612 [==============================] - 36s 60ms/step - loss: 0.2733 - accuracy: 0.8944
Epoch 5/10
612/612 [==============================] - 37s 60ms/step - loss: 0.2270 - accuracy: 0.9157
Epoch 6/10
612/612 [==============================] - 36s 58ms/step - loss: 0.1902 - accuracy: 0.9304
Epoch 7/10
612/612 [==============================] - 37s 60ms/step - loss: 0.1589 - accuracy: 0.9427
Epoch 8/10
612/612 [==============================] - 37s 61ms/step - loss: 0.1321 - accuracy: 0.9520
Epoch 9/10
612/612 [==============================] - 40s 65ms/step - loss: 0.1118 - accuracy: 0.9600
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.4007 - accuracy:

In [ ]:
#Model 5: 1  RNN - BiLSTM with flatten layer, RMSprop 1e-3

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model18 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model18.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-3),
              metrics=['accuracy',get_f1])

model18.fit(x_train, y_train, batch_size=32, epochs=10)


accuracy18= model18.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy18)


predictions18= model18.predict(x_test)
predictions18=np.argmax(predictions18, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions18))
cm = confusion_matrix(y_test, predictions18)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions18))
print('F1 Score: %.3f' % f1_score(y_test, predictions18))
print('Precision: %.3f' % precision_score(y_test, predictions18))

Epoch 1/10
612/612 [==============================] - 45s 68ms/step - loss: 0.4023 - accuracy: 0.8354 - get_f1: 0.8354
Epoch 2/10
612/612 [==============================] - 37s 61ms/step - loss: 0.1860 - accuracy: 0.9305 - get_f1: 0.9305
Epoch 3/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0768 - accuracy: 0.9763 - get_f1: 0.9763
Epoch 4/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0452 - accuracy: 0.9878 - get_f1: 0.9878
Epoch 5/10
612/612 [==============================] - 38s 61ms/step - loss: 0.0292 - accuracy: 0.9909 - get_f1: 0.9909
Epoch 6/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0237 - accuracy: 0.9922 - get_f1: 0.9922
Epoch 7/10
612/612 [==============================] - 38s 61ms/step - loss: 0.0196 - accuracy: 0.9942 - get_f1: 0.9942
Epoch 8/10
612/612 [==============================] - 38s 61ms/step - loss: 0.0184 - accuracy: 0.9943 - get_f1: 0.9943
Epoch 9/10
612/612 [============================

In [ ]:
#Model 5: 1  RNN - BiLSTM with flatten layer, RMSprop 1e-2

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model19 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model19.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-2),
              metrics=['accuracy',get_f1])

model19.fit(x_train, y_train, batch_size=32, epochs=10)


accuracy19= model19.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy19)


predictions19= model19.predict(x_test)
predictions19=np.argmax(predictions19, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions19))
cm = confusion_matrix(y_test, predictions19)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions19))
print('F1 Score: %.3f' % f1_score(y_test, predictions19))
print('Precision: %.3f' % precision_score(y_test, predictions19))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 40s 60ms/step - loss: 0.4724 - accuracy: 0.8183 - get_f1: 0.8183
Epoch 2/10
612/612 [==============================] - 37s 60ms/step - loss: 0.3628 - accuracy: 0.8594 - get_f1: 0.8594
Epoch 3/10
612/612 [==============================] - 36s 60ms/step - loss: 0.2997 - accuracy: 0.8835 - get_f1: 0.8836
Epoch 4/10
612/612 [==============================] - 41s 67ms/step - loss: 0.2603 - accuracy: 0.9033 - get_f1: 0.9034
Epoch 5/10
612/612 [==============================] - 36s 60ms/step - loss: 0.2286 - accuracy: 0.9189 - get_f1: 0.9188
Epoch 6/10
612/612 [==============================] - 36s 60ms/step - loss: 0.2031 - accuracy: 0.9327 - get_f1: 0.9327
Epoch 7/10
612/612 [==============================] - 36s 60ms/step - loss: 0.1812 - accuracy: 0.9401 - get_f1: 0.9400
Epoch 8/10
612/612 [==============================] - 37s 61ms/step - loss: 0.1698 - accuracy: 0.9485 - get_f1: 0.9485
Epoch 9/10
612/612 [==============================] - 37s 6

In [ ]:
#Model 5: 1  RNN - BiLSTM with flatten layer, RMSprop 1e-1

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model20 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model20.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-1),
              metrics=['accuracy',get_f1])

model20.fit(x_train, y_train, batch_size=32, epochs=10)



Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 46s 68ms/step - loss: 0.8715 - accuracy: 0.8007 - get_f1: 0.8008
Epoch 2/10
612/612 [==============================] - 38s 62ms/step - loss: 0.6393 - accuracy: 0.8124 - get_f1: 0.8124
Epoch 3/10
612/612 [==============================] - 37s 61ms/step - loss: 0.5863 - accuracy: 0.8130 - get_f1: 0.8131
Epoch 4/10
612/612 [==============================] - 37s 61ms/step - loss: 0.5321 - accuracy: 0.8190 - get_f1: 0.8190
Epoch 5/10
612/612 [==============================] - 37s 61ms/step - loss: 0.5078 - accuracy: 0.8191 - get_f1: 0.8191
Epoch 6/10
612/612 [==============================] - 37s 61ms/step - loss: 0.4864 - accuracy: 0.8204 - get_f1: 0.8204
Epoch 7/10
612/612 [==============================] - 37s 60ms/step - loss: 0.4943 - accuracy: 0.8199 - get_f1: 0.8199
Epoch 8/10
612/612 [==============================] - 37s 61ms/step - loss: 0.4970 - accuracy: 0.8197 - get_f1: 0.8198
Epoch 9/10
612/612 [==============================] - 37s 6

ValueError: ignored

In [ ]:

accuracy20= model20.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy20)


predictions20= model20.predict(x_test)
predictions20=np.argmax(predictions20, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions20))
cm = confusion_matrix(y_test, predictions20)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions20))
print('F1 Score: %.3f' % f1_score(y_test, predictions20))
print('Precision: %.3f' % precision_score(y_test, predictions20))

204/204 [==============================] - 4s 14ms/step - loss: 0.4827 - accuracy: 0.8202 - get_f1: 0.8201
test accuracy is:  [0.48267966508865356, 0.8201747536659241, 0.8201422095298767]
204/204 [==============================] - 4s 12ms/step
Recall: 0.000
The confusion matrix is [[5350    0]
 [1173    0]]
Accuracy: 0.820
F1 Score: 0.000
Precision: 0.000


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
#Model 5: 1  RNN - BiLSTM with flatten layer, Nadam 1e-4

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model23 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model23.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Nadam(1e-4),
              metrics=['accuracy'])

model23.fit(x_train, y_train, batch_size=32, epochs=10)


accuracy23= model23.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy23)


predictions23= model23.predict(x_test)
predictions23=np.argmax(predictions23, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions23))
cm = confusion_matrix(y_test, predictions23)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions23))
print('F1 Score: %.3f' % f1_score(y_test, predictions23))
print('Precision: %.3f' % precision_score(y_test, predictions23))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 80s 113ms/step - loss: 0.4719 - accuracy: 0.8153
Epoch 2/10
612/612 [==============================] - 69s 112ms/step - loss: 0.3085 - accuracy: 0.8768
Epoch 3/10
612/612 [==============================] - 51s 83ms/step - loss: 0.1695 - accuracy: 0.9396
Epoch 4/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0986 - accuracy: 0.9672
Epoch 5/10
612/612 [==============================] - 39s 63ms/step - loss: 0.0652 - accuracy: 0.9795
Epoch 6/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0476 - accuracy: 0.9854
Epoch 7/10
612/612 [==============================] - 40s 65ms/step - loss: 0.0383 - accuracy: 0.9888
Epoch 8/10
612/612 [==============================] - 38s 63ms/step - loss: 0.0317 - accuracy: 0.9905
Epoch 9/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0283 - accuracy: 0.9917
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.4316 - accurac

In [15]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [16]:
#Model 5: 1  RNN - BiLSTM with flatten layer, Adagrad 1e-4

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model24 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model24.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adagrad(1e-4),
              metrics=['accuracy'])

model24.fit(x_train, y_train, batch_size=32, epochs=10)


accuracy24= model24.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy24)


predictions24= model24.predict(x_test)
predictions24=np.argmax(predictions24, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions24))
cm = confusion_matrix(y_test, predictions24)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions24))
print('F1 Score: %.3f' % f1_score(y_test, predictions24))
print('Precision: %.3f' % precision_score(y_test, predictions24))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 39s 57ms/step - loss: 0.6476 - accuracy: 0.8114
Epoch 2/10
612/612 [==============================] - 35s 57ms/step - loss: 0.6068 - accuracy: 0.8211
Epoch 3/10
612/612 [==============================] - 35s 57ms/step - loss: 0.5753 - accuracy: 0.8211
Epoch 4/10
612/612 [==============================] - 37s 61ms/step - loss: 0.5517 - accuracy: 0.8211
Epoch 5/10
612/612 [==============================] - 35s 58ms/step - loss: 0.5344 - accuracy: 0.8211
Epoch 6/10
612/612 [==============================] - 35s 57ms/step - loss: 0.5217 - accuracy: 0.8211
Epoch 7/10
612/612 [==============================] - 35s 58ms/step - loss: 0.5112 - accuracy: 0.8211
Epoch 8/10
612/612 [==============================] - 37s 61ms/step - loss: 0.5038 - accuracy: 0.8211
Epoch 9/10
612/612 [==============================] - 35s 58ms/step - loss: 0.4983 - accuracy: 0.8211
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.4864 - accuracy:

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [18]:
#Model 5: 1  RNN - BiLSTM with flatten layer, Adadelta 1e-4

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model25 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model25.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adadelta(1e-4),
              metrics=['accuracy'])

model25.fit(x_train, y_train, batch_size=32, epochs=10)


accuracy25= model25.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy25)


predictions25= model25.predict(x_test)
predictions25=np.argmax(predictions25, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions25))
cm = confusion_matrix(y_test, predictions25)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions25))
print('F1 Score: %.3f' % f1_score(y_test, predictions25))
print('Precision: %.3f' % precision_score(y_test, predictions25))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 39s 58ms/step - loss: 0.7040 - accuracy: 0.2402
Epoch 2/10
612/612 [==============================] - 38s 61ms/step - loss: 0.6979 - accuracy: 0.3780
Epoch 3/10
612/612 [==============================] - 36s 58ms/step - loss: 0.6913 - accuracy: 0.5483
Epoch 4/10
612/612 [==============================] - 36s 58ms/step - loss: 0.6836 - accuracy: 0.6870
Epoch 5/10
612/612 [==============================] - 35s 58ms/step - loss: 0.6761 - accuracy: 0.7507
Epoch 6/10
612/612 [==============================] - 35s 58ms/step - loss: 0.6689 - accuracy: 0.7799
Epoch 7/10
612/612 [==============================] - 38s 62ms/step - loss: 0.6624 - accuracy: 0.7971
Epoch 8/10
612/612 [==============================] - 35s 58ms/step - loss: 0.6562 - accuracy: 0.8070
Epoch 9/10
612/612 [==============================] - 36s 58ms/step - loss: 0.6512 - accuracy: 0.8122
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.6429 - accuracy:

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [20]:
#Model 5: 1  RNN - BiLSTM with flatten layer, Adamax 1e-4

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model26 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model26.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adamax(1e-4),
              metrics=['accuracy'])

model26.fit(x_train, y_train, batch_size=32, epochs=10)


accuracy26= model26.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy26)


predictions26= model26.predict(x_test)
predictions26=np.argmax(predictions26, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions26))
cm = confusion_matrix(y_test, predictions26)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions26))
print('F1 Score: %.3f' % f1_score(y_test, predictions26))
print('Precision: %.3f' % precision_score(y_test, predictions26))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 41s 61ms/step - loss: 0.4990 - accuracy: 0.8159
Epoch 2/10
612/612 [==============================] - 35s 58ms/step - loss: 0.4669 - accuracy: 0.8193
Epoch 3/10
612/612 [==============================] - 35s 58ms/step - loss: 0.4578 - accuracy: 0.8194
Epoch 4/10
612/612 [==============================] - 35s 57ms/step - loss: 0.4393 - accuracy: 0.8204
Epoch 5/10
612/612 [==============================] - 37s 61ms/step - loss: 0.4115 - accuracy: 0.8281
Epoch 6/10
612/612 [==============================] - 35s 57ms/step - loss: 0.3891 - accuracy: 0.8379
Epoch 7/10
612/612 [==============================] - 35s 57ms/step - loss: 0.3687 - accuracy: 0.8457
Epoch 8/10
612/612 [==============================] - 35s 57ms/step - loss: 0.3442 - accuracy: 0.8550
Epoch 9/10
612/612 [==============================] - 35s 58ms/step - loss: 0.3220 - accuracy: 0.8671
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.3741 - accuracy:

# Activation Functions Tuning

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
# RNN - BiLSTM with flatten layer, Adam(1e-4), (Softplus+softmax)

from tensorflow.keras import layers
model10 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='softplus'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model10.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model10.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy10= model10.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy10)

predictions10= model10.predict(x_test)

predictions10=np.argmax(predictions10, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions10)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions10))
print('Recall: %.3f' % recall_score(y_test, predictions10))
print('Precision: %.3f' % precision_score(y_test, predictions10))
print('F1 Score: %.3f' % f1_score(y_test, predictions10))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 54s 83ms/step - loss: 0.5304 - accuracy: 0.8085
Epoch 2/10
612/612 [==============================] - 39s 63ms/step - loss: 0.3556 - accuracy: 0.8594
Epoch 3/10
612/612 [==============================] - 38s 63ms/step - loss: 0.1815 - accuracy: 0.9389
Epoch 4/10
612/612 [==============================] - 39s 63ms/step - loss: 0.1045 - accuracy: 0.9687
Epoch 5/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0686 - accuracy: 0.9805
Epoch 6/10
612/612 [==============================] - 39s 63ms/step - loss: 0.0512 - accuracy: 0.9861
Epoch 7/10
612/612 [==============================] - 39s 63ms/step - loss: 0.0407 - accuracy: 0.9884
Epoch 8/10
612/612 [==============================] - 42s 69ms/step - loss: 0.0325 - accuracy: 0.9906
Epoch 9/10
612/612 [==============================] - 39s 64ms/step - loss: 0.0289 - accuracy: 0.9912
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.4403 - accuracy:

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
# RNN - BiLSTM with flatten layer, Adam(1e-4), (Softsign+softmax)

from tensorflow.keras import layers
model11 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='softsign'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model11.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model11.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy11= model11.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy11)

predictions11= model11.predict(x_test)

predictions11=np.argmax(predictions11, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions11)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions11))
print('Recall: %.3f' % recall_score(y_test, predictions11))
print('Precision: %.3f' % precision_score(y_test, predictions11))
print('F1 Score: %.3f' % f1_score(y_test, predictions11))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 42s 62ms/step - loss: 0.4468 - accuracy: 0.8238
Epoch 2/10
612/612 [==============================] - 38s 63ms/step - loss: 0.2418 - accuracy: 0.9056
Epoch 3/10
612/612 [==============================] - 38s 62ms/step - loss: 0.1112 - accuracy: 0.9628
Epoch 4/10
612/612 [==============================] - 39s 63ms/step - loss: 0.0595 - accuracy: 0.9811
Epoch 5/10
612/612 [==============================] - 39s 64ms/step - loss: 0.0428 - accuracy: 0.9880
Epoch 6/10
612/612 [==============================] - 44s 71ms/step - loss: 0.0314 - accuracy: 0.9907
Epoch 7/10
612/612 [==============================] - 40s 65ms/step - loss: 0.0278 - accuracy: 0.9922
Epoch 8/10
612/612 [==============================] - 39s 64ms/step - loss: 0.0236 - accuracy: 0.9929
Epoch 9/10
612/612 [==============================] - 39s 64ms/step - loss: 0.0208 - accuracy: 0.9934
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.3777 - accuracy:

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
# RNN - BiLSTM with flatten layer, Adam(1e-4), (tanh+softmax)

from tensorflow.keras import layers
model12 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model12.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model12.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy12= model12.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy12)

predictions12= model12.predict(x_test)

predictions12=np.argmax(predictions12, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions12)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions12))
print('Recall: %.3f' % recall_score(y_test, predictions12))
print('Precision: %.3f' % precision_score(y_test, predictions12))
print('F1 Score: %.3f' % f1_score(y_test, predictions12))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 42s 64ms/step - loss: 0.4439 - accuracy: 0.8231
Epoch 2/10
612/612 [==============================] - 43s 71ms/step - loss: 0.2406 - accuracy: 0.9060
Epoch 3/10
612/612 [==============================] - 39s 63ms/step - loss: 0.1098 - accuracy: 0.9630
Epoch 4/10
612/612 [==============================] - 38s 63ms/step - loss: 0.0607 - accuracy: 0.9803
Epoch 5/10
612/612 [==============================] - 39s 63ms/step - loss: 0.0393 - accuracy: 0.9891
Epoch 6/10
612/612 [==============================] - 39s 63ms/step - loss: 0.0334 - accuracy: 0.9902
Epoch 7/10
612/612 [==============================] - 47s 76ms/step - loss: 0.0269 - accuracy: 0.9920
Epoch 8/10
612/612 [==============================] - 39s 64ms/step - loss: 0.0234 - accuracy: 0.9927
Epoch 9/10
612/612 [==============================] - 40s 65ms/step - loss: 0.0203 - accuracy: 0.9926
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.3459 - accuracy:

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
# RNN - BiLSTM with flatten layer, Adam(1e-4), (selu+softmax)

from tensorflow.keras import layers
model13 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='selu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model13.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model13.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy13= model13.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy13)

predictions13= model13.predict(x_test)

predictions13=np.argmax(predictions13, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions13)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions13))
print('Recall: %.3f' % recall_score(y_test, predictions13))
print('Precision: %.3f' % precision_score(y_test, predictions13))
print('F1 Score: %.3f' % f1_score(y_test, predictions13))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 41s 62ms/step - loss: 0.4430 - accuracy: 0.8253
Epoch 2/10
612/612 [==============================] - 38s 62ms/step - loss: 0.2334 - accuracy: 0.9093
Epoch 3/10
612/612 [==============================] - 39s 63ms/step - loss: 0.1031 - accuracy: 0.9647
Epoch 4/10
612/612 [==============================] - 38s 63ms/step - loss: 0.0568 - accuracy: 0.9827
Epoch 5/10
612/612 [==============================] - 38s 63ms/step - loss: 0.0387 - accuracy: 0.9889
Epoch 6/10
612/612 [==============================] - 38s 63ms/step - loss: 0.0303 - accuracy: 0.9917
Epoch 7/10
612/612 [==============================] - 42s 68ms/step - loss: 0.0267 - accuracy: 0.9916
Epoch 8/10
612/612 [==============================] - 38s 61ms/step - loss: 0.0246 - accuracy: 0.9926
Epoch 9/10
612/612 [==============================] - 38s 61ms/step - loss: 0.0210 - accuracy: 0.9929
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.4060 - accuracy:

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
# RNN - BiLSTM with flatten layer, Adam(1e-4), (elu+softmax)

from tensorflow.keras import layers
model14 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model14.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model14.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy14= model14.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy14)

predictions14= model14.predict(x_test)

predictions14=np.argmax(predictions14, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions14)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions14))
print('Recall: %.3f' % recall_score(y_test, predictions14))
print('Precision: %.3f' % precision_score(y_test, predictions14))
print('F1 Score: %.3f' % f1_score(y_test, predictions14))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 40s 60ms/step - loss: 0.4447 - accuracy: 0.8245
Epoch 2/10
612/612 [==============================] - 37s 61ms/step - loss: 0.2389 - accuracy: 0.9074
Epoch 3/10
612/612 [==============================] - 40s 66ms/step - loss: 0.1104 - accuracy: 0.9616
Epoch 4/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0626 - accuracy: 0.9799
Epoch 5/10
612/612 [==============================] - 37s 61ms/step - loss: 0.0417 - accuracy: 0.9871
Epoch 6/10
612/612 [==============================] - 37s 60ms/step - loss: 0.0312 - accuracy: 0.9906
Epoch 7/10
612/612 [==============================] - 37s 61ms/step - loss: 0.0273 - accuracy: 0.9911
Epoch 8/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0229 - accuracy: 0.9913
Epoch 9/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0192 - accuracy: 0.9922
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.4510 - accuracy:

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
# RNN - BiLSTM with flatten layer, Adam(1e-4), (exponential+softmax)

from tensorflow.keras import layers
model15 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='exponential'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model15.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model15.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy15= model15.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy15)

predictions15= model15.predict(x_test)

predictions15=np.argmax(predictions15, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions15)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions15))
print('Recall: %.3f' % recall_score(y_test, predictions15))
print('Precision: %.3f' % precision_score(y_test, predictions15))
print('F1 Score: %.3f' % f1_score(y_test, predictions15))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 45s 68ms/step - loss: 0.5798 - accuracy: 0.7910
Epoch 2/10
612/612 [==============================] - 38s 62ms/step - loss: 0.3381 - accuracy: 0.8741
Epoch 3/10
612/612 [==============================] - 38s 63ms/step - loss: 0.1578 - accuracy: 0.9472
Epoch 4/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0892 - accuracy: 0.9720
Epoch 5/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0588 - accuracy: 0.9827
Epoch 6/10
612/612 [==============================] - 39s 63ms/step - loss: 0.0438 - accuracy: 0.9878
Epoch 7/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0352 - accuracy: 0.9902
Epoch 8/10
612/612 [==============================] - 38s 62ms/step - loss: 0.0307 - accuracy: 0.9915
Epoch 9/10
612/612 [==============================] - 42s 69ms/step - loss: 0.0266 - accuracy: 0.9920
Epoch 10/10
204/204 [==============================] - 3s 9ms/step - loss: 0.4391 - accuracy:

#Batch Size Tuning 


In [ ]:
#RNN - BiLSTM with flatten (10 Batch Size)

from tensorflow.keras import layers
model16 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model16.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model16.fit(x_train, y_train, batch_size=10, epochs=10)


predictions16= model16.predict(x_test)
predictions16=np.argmax(predictions16, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions16)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions16))
print('Recall: %.3f' % recall_score(y_test, predictions16))
print('Precision: %.3f' % precision_score(y_test, predictions16))
print('F1 Score: %.3f' % f1_score(y_test, predictions16))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1958/1958 [==============================] - 174s 86ms/step - loss: 0.4294 - accuracy: 0.8299
Epoch 2/10
1958/1958 [==============================] - 126s 64ms/step - loss: 0.2031 - accuracy: 0.9239
Epoch 3/10
1958/1958 [==============================] - 122s 62ms/step - loss: 0.0810 - accuracy: 0.9739
Epoch 4/10
1958/1958 [==============================] - 125s 64ms/step - loss: 0.0435 - accuracy: 0.9883
Epoch 5/10
1958/1958 [==============================] - 122s 62ms/step - loss: 0.0334 - accuracy: 0.9910
Epoch 6/10
1958/1958 [==============================] - 125s 64ms/step - loss: 0.0244 - accuracy: 0.9930
Epoch 7/10
1958/1958 [==============================] - 121s 62ms/step - loss: 0.0207 - accuracy: 0.9934
Epoch 8/10
1958/1958 [==============================] - 125s 64ms/step - loss: 0.0170 - accuracy: 0.9939
Epoch 9/10
1958/1958 [==============================] - 121s 62ms/step - loss: 0.0140 - accuracy: 0.9938
Epoch 10/10
204/204 [==============================] - 3s 8ms/step

In [ ]:
#RNN - BiLSTM with flatten (40 Batch Size)

from tensorflow.keras import layers
model17 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model17.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model17.fit(x_train, y_train, batch_size=40, epochs=10)


predictions17= model17.predict(x_test)
predictions17=np.argmax(predictions17, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions17)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions17))
print('Recall: %.3f' % recall_score(y_test, predictions17))
print('Precision: %.3f' % precision_score(y_test, predictions17))
print('F1 Score: %.3f' % f1_score(y_test, predictions17))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


490/490 [==============================] - 38s 71ms/step - loss: 0.4679 - accuracy: 0.8171
Epoch 2/10
490/490 [==============================] - 31s 63ms/step - loss: 0.2832 - accuracy: 0.8903
Epoch 3/10
490/490 [==============================] - 31s 62ms/step - loss: 0.1394 - accuracy: 0.9524
Epoch 4/10
490/490 [==============================] - 30s 62ms/step - loss: 0.0730 - accuracy: 0.9766
Epoch 5/10
490/490 [==============================] - 47s 95ms/step - loss: 0.0463 - accuracy: 0.9872
Epoch 6/10
490/490 [==============================] - 35s 71ms/step - loss: 0.0376 - accuracy: 0.9899
Epoch 7/10
490/490 [==============================] - 31s 63ms/step - loss: 0.0296 - accuracy: 0.9919
Epoch 8/10
490/490 [==============================] - 31s 63ms/step - loss: 0.0262 - accuracy: 0.9925
Epoch 9/10
490/490 [==============================] - 31s 64ms/step - loss: 0.0247 - accuracy: 0.9924
Epoch 10/10
204/204 [==============================] - 2s 8ms/step
The Confusion Matrix is [[

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#RNN - BiLSTM with flatten (60 Batch Size)

from tensorflow.keras import layers
model18 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model18.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model18.fit(x_train, y_train, batch_size=60, epochs=10)


predictions18= model18.predict(x_test)
predictions18=np.argmax(predictions18, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions18)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions18))
print('Recall: %.3f' % recall_score(y_test, predictions18))
print('Precision: %.3f' % precision_score(y_test, predictions18))
print('F1 Score: %.3f' % f1_score(y_test, predictions18))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


327/327 [==============================] - 23s 62ms/step - loss: 0.4927 - accuracy: 0.8154
Epoch 2/10
327/327 [==============================] - 20s 61ms/step - loss: 0.3373 - accuracy: 0.8640
Epoch 3/10
327/327 [==============================] - 20s 62ms/step - loss: 0.1742 - accuracy: 0.9399
Epoch 4/10
327/327 [==============================] - 20s 61ms/step - loss: 0.0971 - accuracy: 0.9701
Epoch 5/10
327/327 [==============================] - 20s 61ms/step - loss: 0.0620 - accuracy: 0.9812
Epoch 6/10
327/327 [==============================] - 20s 61ms/step - loss: 0.0452 - accuracy: 0.9872
Epoch 7/10
327/327 [==============================] - 20s 62ms/step - loss: 0.0375 - accuracy: 0.9898
Epoch 8/10
327/327 [==============================] - 20s 62ms/step - loss: 0.0319 - accuracy: 0.9910
Epoch 9/10
327/327 [==============================] - 20s 61ms/step - loss: 0.0281 - accuracy: 0.9921
Epoch 10/10
204/204 [==============================] - 2s 8ms/step
The Confusion Matrix is [[

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#RNN - BiLSTM with flatten (80 Batch Size)

from tensorflow.keras import layers
model19 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model19.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model19.fit(x_train, y_train, batch_size=80, epochs=10)


predictions19= model19.predict(x_test)
predictions19=np.argmax(predictions19, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions19)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions19))
print('Recall: %.3f' % recall_score(y_test, predictions19))
print('Precision: %.3f' % precision_score(y_test, predictions19))
print('F1 Score: %.3f' % f1_score(y_test, predictions19))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


245/245 [==============================] - 18s 61ms/step - loss: 0.4954 - accuracy: 0.8167
Epoch 2/10
245/245 [==============================] - 15s 62ms/step - loss: 0.3656 - accuracy: 0.8507
Epoch 3/10
245/245 [==============================] - 15s 62ms/step - loss: 0.2044 - accuracy: 0.9263
Epoch 4/10
245/245 [==============================] - 15s 62ms/step - loss: 0.1143 - accuracy: 0.9635
Epoch 5/10
245/245 [==============================] - 15s 61ms/step - loss: 0.0747 - accuracy: 0.9775
Epoch 6/10
245/245 [==============================] - 15s 63ms/step - loss: 0.0552 - accuracy: 0.9837
Epoch 7/10
245/245 [==============================] - 15s 61ms/step - loss: 0.0437 - accuracy: 0.9873
Epoch 8/10
245/245 [==============================] - 15s 62ms/step - loss: 0.0383 - accuracy: 0.9894
Epoch 9/10
245/245 [==============================] - 15s 62ms/step - loss: 0.0325 - accuracy: 0.9912
Epoch 10/10
204/204 [==============================] - 3s 13ms/step
The Confusion Matrix is [

# Epochs

In [ ]:
#RNN - BiLSTM with flatten (25 Epochs)

from tensorflow.keras import layers
model20 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model20.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model20.fit(x_train, y_train, batch_size=32, epochs=25)

Epoch 1/25


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 45s 63ms/step - loss: 0.4673 - accuracy: 0.8189
Epoch 2/25
612/612 [==============================] - 41s 68ms/step - loss: 0.2670 - accuracy: 0.8991
Epoch 3/25
612/612 [==============================] - 70s 115ms/step - loss: 0.1261 - accuracy: 0.9592
Epoch 4/25
612/612 [==============================] - 58s 95ms/step - loss: 0.0711 - accuracy: 0.9782
Epoch 5/25
612/612 [==============================] - 45s 73ms/step - loss: 0.0483 - accuracy: 0.9864
Epoch 6/25
612/612 [==============================] - 40s 65ms/step - loss: 0.0379 - accuracy: 0.9892
Epoch 7/25
612/612 [==============================] - 40s 66ms/step - loss: 0.0317 - accuracy: 0.9915
Epoch 8/25
612/612 [==============================] - 40s 65ms/step - loss: 0.0282 - accuracy: 0.9922
Epoch 9/25
612/612 [==============================] - 40s 65ms/step - loss: 0.0242 - accuracy: 0.9926
Epoch 10/25
612/612 [==============================] - 39s 64ms/step - loss: 0.0218 - accura

AxisError: ignored

In [ ]:

predictions20= model20.predict(x_test)
predictions20=np.argmax(predictions20, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions20)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions20))
print('Recall: %.3f' % recall_score(y_test, predictions20))
print('Precision: %.3f' % precision_score(y_test, predictions20))
print('F1 Score: %.3f' % f1_score(y_test, predictions20))

204/204 [==============================] - 2s 8ms/step
The Confusion Matrix is [[5226   98]
 [  90 1109]]
Accuracy: 0.971
Recall: 0.925
Precision: 0.919
F1 Score: 0.922


In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#RNN - BiLSTM with flatten (50 Epochs)

from tensorflow.keras import layers
model21 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model21.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model21.fit(x_train, y_train, batch_size=32, epochs=50)


predictions21= model21.predict(x_test)
predictions21=np.argmax(predictions21, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions21)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions21))
print('Recall: %.3f' % recall_score(y_test, predictions21))
print('Precision: %.3f' % precision_score(y_test, predictions21))
print('F1 Score: %.3f' % f1_score(y_test, predictions21))

Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 127s 200ms/step - loss: 0.4601 - accuracy: 0.8218
Epoch 2/50
612/612 [==============================] - 122s 200ms/step - loss: 0.2673 - accuracy: 0.8976
Epoch 3/50
612/612 [==============================] - 122s 199ms/step - loss: 0.1246 - accuracy: 0.9586
Epoch 4/50
612/612 [==============================] - 122s 199ms/step - loss: 0.0673 - accuracy: 0.9797
Epoch 5/50
612/612 [==============================] - 121s 198ms/step - loss: 0.0453 - accuracy: 0.9864
Epoch 6/50
612/612 [==============================] - 122s 199ms/step - loss: 0.0365 - accuracy: 0.9901
Epoch 7/50
612/612 [==============================] - 122s 200ms/step - loss: 0.0289 - accuracy: 0.9919
Epoch 8/50
612/612 [==============================] - 121s 198ms/step - loss: 0.0256 - accuracy: 0.9925
Epoch 9/50
612/612 [==============================] - 123s 202ms/step - loss: 0.0235 - accuracy: 0.9922
Epoch 10/50
612/612 [==============================] - 123s 201ms/step - lo

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#RNN - BiLSTM with flatten (75 epochs)

from tensorflow.keras import layers
model22 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])

model22.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model22.fit(x_train, y_train, batch_size=32, epochs=75)


predictions22= model22.predict(x_test)
predictions22=np.argmax(predictions22, axis=1)
y_test=np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test, predictions22)
print('The Confusion Matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions22))
print('Recall: %.3f' % recall_score(y_test, predictions22))
print('Precision: %.3f' % precision_score(y_test, predictions22))
print('F1 Score: %.3f' % f1_score(y_test, predictions22))

Epoch 1/75
612/612 [==============================] - 57s 78ms/step - loss: 0.4702 - accuracy: 0.8156
Epoch 2/75
612/612 [==============================] - 40s 65ms/step - loss: 0.2835 - accuracy: 0.8888
Epoch 3/75
612/612 [==============================] - 39s 63ms/step - loss: 0.1388 - accuracy: 0.9537
Epoch 4/75
612/612 [==============================] - 40s 65ms/step - loss: 0.0768 - accuracy: 0.9767
Epoch 5/75
612/612 [==============================] - 37s 61ms/step - loss: 0.0523 - accuracy: 0.9850
Epoch 6/75
612/612 [==============================] - 40s 65ms/step - loss: 0.0400 - accuracy: 0.9887
Epoch 7/75
612/612 [==============================] - 38s 62ms/step - loss: 0.0332 - accuracy: 0.9905
Epoch 8/75
612/612 [==============================] - 40s 65ms/step - loss: 0.0283 - accuracy: 0.9921
Epoch 9/75
612/612 [==============================] - 38s 61ms/step - loss: 0.0262 - accuracy: 0.9925
Epoch 10/75
612/612 [==============================] - 39s 64ms/step - loss: 0.024